# Acquire Walkthrough
---

## Lets import our libraries

In [1]:
import pandas as pd
from pandas_profiling import ProfileReport

import env
import acquire
import prep

## Next lets bring in our zillow data from the SQL server

***If reproducing, run the commented code below***

In [2]:
# query = '''
# SELECT p.id, p.bathroomcnt, p.bedroomcnt, p.calculatedfinishedsquarefeet, 
#     p.fips, p.taxvaluedollarcnt, p.taxamount, 
#     ROUND((p.taxamount / p.taxvaluedollarcnt) * 100, 2) AS taxrate
# FROM properties_2017 AS p
# JOIN predictions_2017 AS pr USING (parcelid)
# WHERE p.propertylandusetypeid IN (261, 262, 263, 264, 266, 268, 273, 276, 279)
# AND pr.transactiondate BETWEEN "2017-05-01" AND '2017-06-30';
# '''

# url = acquire.get_url('zillow')

# zillow = pd.read_sql(query, url, index_col='id')

## Now lets export is as a csv and bring it back so that we don't have to rerun the query each time.

In [3]:
# zillow.to_csv('zillow.csv')

In [4]:
zillow = pd.read_csv('zillow.csv')
zillow

,id,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,taxvaluedollarcnt,taxamount,taxrate
0,2061546,2.0,3.0,1458.0,6037.0,136104.0,2319.90,1.70
1,1834372,1.0,2.0,1421.0,6037.0,35606.0,543.69,1.53
2,1923117,3.0,4.0,2541.0,6059.0,880456.0,9819.72,1.12
3,2121349,2.0,3.0,1650.0,6037.0,614000.0,7673.19,1.25
4,2093710,1.0,2.0,693.0,6037.0,274237.0,3267.47,1.19
...,...,...,...,...,...,...,...,...
20343,2922089,2.0,2.0,1030.0,6037.0,359829.0,4378.59,1.22
20344,1948691,2.0,3.0,1536.0,6037.0,297097.0,3519.78,1.18
20345,444575,3.0,5.0,2655.0,6059.0,746963.0,8065.50,1.08
20346,1480299,3.0,4.0,2305.0,6037.0,579047.0,6996.21,1.21


## Great, now we need to bring in some outside data so that we can identify the county by the fips column in zillow

FIPS data was copied from this location: https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697

***If reproducing, run the below commented code to create the file in your directory***

In [5]:
#commented after it ran
#fips = pd.read_clipboard().set_index('FIPS')

In [6]:
#commented after it ran
#fips.to_csv('fips.csv')

In [7]:
FIPS = pd.read_csv('fips.csv')
FIPS

,FIPS,Name,State
0,1001,Autauga,AL
1,1003,Baldwin,AL
2,1005,Barbour,AL
3,1007,Bibb,AL
4,1009,Blount,AL
...,...,...,...
3227,72151,Yabucoa,PR
3228,72153,Yauco,PR
3229,78010,St. Croix,VI
3230,78020,St. John,VI


## Now lets merge the tables so we have it all on one

In [8]:
zillow = zillow.merge(FIPS, left_on='fips', right_on='FIPS')

In [9]:
zillow

,id,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,taxvaluedollarcnt,taxamount,taxrate,FIPS,Name,State
0,2061546,2.0,3.0,1458.0,6037.0,136104.0,2319.90,1.70,6037,Los Angeles,CA
1,1834372,1.0,2.0,1421.0,6037.0,35606.0,543.69,1.53,6037,Los Angeles,CA
2,2121349,2.0,3.0,1650.0,6037.0,614000.0,7673.19,1.25,6037,Los Angeles,CA
3,2093710,1.0,2.0,693.0,6037.0,274237.0,3267.47,1.19,6037,Los Angeles,CA
4,496818,0.0,0.0,1378.0,6037.0,168828.0,2135.39,1.26,6037,Los Angeles,CA
...,...,...,...,...,...,...,...,...,...,...,...
20343,1624472,3.0,3.0,2632.0,6111.0,795992.0,8449.92,1.06,6111,Ventura,CA
20344,997928,3.0,4.0,2734.0,6111.0,897000.0,9585.08,1.07,6111,Ventura,CA
20345,849307,2.5,4.0,2314.0,6111.0,485364.0,9549.08,1.97,6111,Ventura,CA
20346,2416171,2.0,2.0,1178.0,6111.0,169908.0,1779.86,1.05,6111,Ventura,CA


## Now lets get a quick look at the data

In [10]:
zillow.describe()

,id,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,taxvaluedollarcnt,taxamount,taxrate,FIPS
count,2.034800e+04,20348.000000,20348.000000,20303.000000,20348.000000,2.034800e+04,20347.000000,20347.000000,20348.000000
mean,1.510457e+06,2.285507,3.021181,1776.690391,6049.483782,5.056470e+05,6158.269147,1.315918,6049.483782
std,8.598606e+05,0.957624,1.029851,936.896616,20.960930,6.586031e+05,7642.769432,0.602459,20.960930
min,3.490000e+02,0.000000,0.000000,242.000000,6037.000000,1.050400e+04,19.920000,0.010000,6037.000000
25%,7.746850e+05,2.000000,2.000000,1176.000000,6037.000000,2.107685e+05,2736.860000,1.160000,6037.000000
50%,1.525712e+06,2.000000,3.000000,1536.000000,6037.000000,3.670000e+05,4525.660000,1.220000,6037.000000
75%,2.253321e+06,3.000000,4.000000,2103.000000,6059.000000,5.895145e+05,7132.225000,1.340000,6059.000000
max,2.982274e+06,11.000000,12.000000,15450.000000,6111.000000,2.385837e+07,276797.830000,45.290000,6111.000000


We can see that their are some zero in bedroom and bathroom count so we will probably go ahead and get rid of those

In [11]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20348 entries, 0 to 20347
Data columns (total 11 columns):
id                              20348 non-null int64
bathroomcnt                     20348 non-null float64
bedroomcnt                      20348 non-null float64
calculatedfinishedsquarefeet    20303 non-null float64
fips                            20348 non-null float64
taxvaluedollarcnt               20348 non-null float64
taxamount                       20347 non-null float64
taxrate                         20347 non-null float64
FIPS                            20348 non-null int64
Name                            20348 non-null object
State                           20348 non-null object
dtypes: float64(7), int64(2), object(2)
memory usage: 1.9+ MB


## Now lets build a function that reads these csvs

In [12]:
zillow = acquire.get_data()
zillow

,id,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,taxvaluedollarcnt,taxamount,taxrate,FIPS,Name,State
0,2061546,2.0,3.0,1458.0,6037.0,136104.0,2319.90,1.70,6037,Los Angeles,CA
1,1834372,1.0,2.0,1421.0,6037.0,35606.0,543.69,1.53,6037,Los Angeles,CA
2,2121349,2.0,3.0,1650.0,6037.0,614000.0,7673.19,1.25,6037,Los Angeles,CA
3,2093710,1.0,2.0,693.0,6037.0,274237.0,3267.47,1.19,6037,Los Angeles,CA
4,496818,0.0,0.0,1378.0,6037.0,168828.0,2135.39,1.26,6037,Los Angeles,CA
...,...,...,...,...,...,...,...,...,...,...,...
20343,1624472,3.0,3.0,2632.0,6111.0,795992.0,8449.92,1.06,6111,Ventura,CA
20344,997928,3.0,4.0,2734.0,6111.0,897000.0,9585.08,1.07,6111,Ventura,CA
20345,849307,2.5,4.0,2314.0,6111.0,485364.0,9549.08,1.97,6111,Ventura,CA
20346,2416171,2.0,2.0,1178.0,6111.0,169908.0,1779.86,1.05,6111,Ventura,CA


# Preparation Walkthrough

## First lets check for duplicates

In [13]:
zillow[zillow.duplicated(keep=False)]

,id,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,taxvaluedollarcnt,taxamount,taxrate,FIPS,Name,State
1219,159239,3.0,2.0,1149.0,6037.0,218619.0,3074.84,1.41,6037,Los Angeles,CA
1220,159239,3.0,2.0,1149.0,6037.0,218619.0,3074.84,1.41,6037,Los Angeles,CA
7194,1968735,6.0,12.0,8469.0,6037.0,2485282.0,30342.87,1.22,6037,Los Angeles,CA
7195,1968735,6.0,12.0,8469.0,6037.0,2485282.0,30342.87,1.22,6037,Los Angeles,CA
7196,1968735,6.0,12.0,8469.0,6037.0,2485282.0,30342.87,1.22,6037,Los Angeles,CA
12841,2011553,1.5,2.0,1190.0,6059.0,357000.0,3772.90,1.06,6059,Orange,CA
12842,2011553,1.5,2.0,1190.0,6059.0,357000.0,3772.90,1.06,6059,Orange,CA
13317,2456793,1.0,2.0,1025.0,6059.0,48107.0,862.90,1.79,6059,Orange,CA
13318,2456793,1.0,2.0,1025.0,6059.0,48107.0,862.90,1.79,6059,Orange,CA
14821,2353255,2.0,4.0,2126.0,6059.0,98473.0,1435.68,1.46,6059,Orange,CA


### There are a few duplicates but the info is identical for each so lets get rid of the second duplicate for each

In [14]:
zillow = zillow.drop_duplicates()

## Lets see if we have any dtype or null issues

In [15]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20340 entries, 0 to 20347
Data columns (total 11 columns):
id                              20340 non-null int64
bathroomcnt                     20340 non-null float64
bedroomcnt                      20340 non-null float64
calculatedfinishedsquarefeet    20295 non-null float64
fips                            20340 non-null float64
taxvaluedollarcnt               20340 non-null float64
taxamount                       20339 non-null float64
taxrate                         20339 non-null float64
FIPS                            20340 non-null int64
Name                            20340 non-null object
State                           20340 non-null object
dtypes: float64(7), int64(2), object(2)
memory usage: 1.9+ MB


### So there are missing valuesin 3 columns, but not enough to make a difference so lets just drop them for now

In [16]:
zillow = zillow.dropna()
zillow

,id,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,taxvaluedollarcnt,taxamount,taxrate,FIPS,Name,State
0,2061546,2.0,3.0,1458.0,6037.0,136104.0,2319.90,1.70,6037,Los Angeles,CA
1,1834372,1.0,2.0,1421.0,6037.0,35606.0,543.69,1.53,6037,Los Angeles,CA
2,2121349,2.0,3.0,1650.0,6037.0,614000.0,7673.19,1.25,6037,Los Angeles,CA
3,2093710,1.0,2.0,693.0,6037.0,274237.0,3267.47,1.19,6037,Los Angeles,CA
4,496818,0.0,0.0,1378.0,6037.0,168828.0,2135.39,1.26,6037,Los Angeles,CA
...,...,...,...,...,...,...,...,...,...,...,...
20343,1624472,3.0,3.0,2632.0,6111.0,795992.0,8449.92,1.06,6111,Ventura,CA
20344,997928,3.0,4.0,2734.0,6111.0,897000.0,9585.08,1.07,6111,Ventura,CA
20345,849307,2.5,4.0,2314.0,6111.0,485364.0,9549.08,1.97,6111,Ventura,CA
20346,2416171,2.0,2.0,1178.0,6111.0,169908.0,1779.86,1.05,6111,Ventura,CA


In [17]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20294 entries, 0 to 20347
Data columns (total 11 columns):
id                              20294 non-null int64
bathroomcnt                     20294 non-null float64
bedroomcnt                      20294 non-null float64
calculatedfinishedsquarefeet    20294 non-null float64
fips                            20294 non-null float64
taxvaluedollarcnt               20294 non-null float64
taxamount                       20294 non-null float64
taxrate                         20294 non-null float64
FIPS                            20294 non-null int64
Name                            20294 non-null object
State                           20294 non-null object
dtypes: float64(7), int64(2), object(2)
memory usage: 1.9+ MB


### Also lets make sure bedroomcnt needs to be a float

In [18]:
zillow.bedroomcnt.value_counts()

3.0     8220
2.0     5103
4.0     4633
5.0     1162
1.0      874
6.0      183
0.0       86
7.0       27
9.0        2
11.0       1
10.0       1
8.0        1
12.0       1
Name: bedroomcnt, dtype: int64

In [19]:
zillow.bedroomcnt = zillow.bedroomcnt.astype('int')
zillow

,id,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,taxvaluedollarcnt,taxamount,taxrate,FIPS,Name,State
0,2061546,2.0,3,1458.0,6037.0,136104.0,2319.90,1.70,6037,Los Angeles,CA
1,1834372,1.0,2,1421.0,6037.0,35606.0,543.69,1.53,6037,Los Angeles,CA
2,2121349,2.0,3,1650.0,6037.0,614000.0,7673.19,1.25,6037,Los Angeles,CA
3,2093710,1.0,2,693.0,6037.0,274237.0,3267.47,1.19,6037,Los Angeles,CA
4,496818,0.0,0,1378.0,6037.0,168828.0,2135.39,1.26,6037,Los Angeles,CA
...,...,...,...,...,...,...,...,...,...,...,...
20343,1624472,3.0,3,2632.0,6111.0,795992.0,8449.92,1.06,6111,Ventura,CA
20344,997928,3.0,4,2734.0,6111.0,897000.0,9585.08,1.07,6111,Ventura,CA
20345,849307,2.5,4,2314.0,6111.0,485364.0,9549.08,1.97,6111,Ventura,CA
20346,2416171,2.0,2,1178.0,6111.0,169908.0,1779.86,1.05,6111,Ventura,CA


## There are some duplicate columns so lets get rid of those

In [20]:
zillow = zillow.drop(columns='fips')
zillow

,id,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,taxamount,taxrate,FIPS,Name,State
0,2061546,2.0,3,1458.0,136104.0,2319.90,1.70,6037,Los Angeles,CA
1,1834372,1.0,2,1421.0,35606.0,543.69,1.53,6037,Los Angeles,CA
2,2121349,2.0,3,1650.0,614000.0,7673.19,1.25,6037,Los Angeles,CA
3,2093710,1.0,2,693.0,274237.0,3267.47,1.19,6037,Los Angeles,CA
4,496818,0.0,0,1378.0,168828.0,2135.39,1.26,6037,Los Angeles,CA
...,...,...,...,...,...,...,...,...,...,...
20343,1624472,3.0,3,2632.0,795992.0,8449.92,1.06,6111,Ventura,CA
20344,997928,3.0,4,2734.0,897000.0,9585.08,1.07,6111,Ventura,CA
20345,849307,2.5,4,2314.0,485364.0,9549.08,1.97,6111,Ventura,CA
20346,2416171,2.0,2,1178.0,169908.0,1779.86,1.05,6111,Ventura,CA


## Lets also clarify the names of some of the columns

In [21]:
zillow = zillow.rename(columns={'calculatedfinishedsquarefeet': 'squarefeet', 'Name': 'County'})
zillow

,id,bathroomcnt,bedroomcnt,squarefeet,taxvaluedollarcnt,taxamount,taxrate,FIPS,County,State
0,2061546,2.0,3,1458.0,136104.0,2319.90,1.70,6037,Los Angeles,CA
1,1834372,1.0,2,1421.0,35606.0,543.69,1.53,6037,Los Angeles,CA
2,2121349,2.0,3,1650.0,614000.0,7673.19,1.25,6037,Los Angeles,CA
3,2093710,1.0,2,693.0,274237.0,3267.47,1.19,6037,Los Angeles,CA
4,496818,0.0,0,1378.0,168828.0,2135.39,1.26,6037,Los Angeles,CA
...,...,...,...,...,...,...,...,...,...,...
20343,1624472,3.0,3,2632.0,795992.0,8449.92,1.06,6111,Ventura,CA
20344,997928,3.0,4,2734.0,897000.0,9585.08,1.07,6111,Ventura,CA
20345,849307,2.5,4,2314.0,485364.0,9549.08,1.97,6111,Ventura,CA
20346,2416171,2.0,2,1178.0,169908.0,1779.86,1.05,6111,Ventura,CA


## Now lets make a function that gets the data for us and cleans it.

In [22]:
def acquire_and_prep_data():
    zillow = acquire.get_data()

    zillow = zillow.drop_duplicates()
    zillow = zillow.dropna()
    zillow = zillow.drop(columns='fips')
    zillow.bedroomcnt = zillow.bedroomcnt.astype('int')
    zillow = zillow.rename(columns={'calculatedfinishedsquarefeet': 'squarefeet', 'Name': 'County'})

    return zillow

In [23]:
prep.acquire_and_prep_data()

,id,bathroomcnt,bedroomcnt,squarefeet,taxvaluedollarcnt,taxamount,taxrate,FIPS,County,State
0,2061546,2.0,3,1458.0,136104.0,2319.90,1.70,6037,Los Angeles,CA
1,1834372,1.0,2,1421.0,35606.0,543.69,1.53,6037,Los Angeles,CA
2,2121349,2.0,3,1650.0,614000.0,7673.19,1.25,6037,Los Angeles,CA
3,2093710,1.0,2,693.0,274237.0,3267.47,1.19,6037,Los Angeles,CA
4,496818,0.0,0,1378.0,168828.0,2135.39,1.26,6037,Los Angeles,CA
...,...,...,...,...,...,...,...,...,...,...
20343,1624472,3.0,3,2632.0,795992.0,8449.92,1.06,6111,Ventura,CA
20344,997928,3.0,4,2734.0,897000.0,9585.08,1.07,6111,Ventura,CA
20345,849307,2.5,4,2314.0,485364.0,9549.08,1.97,6111,Ventura,CA
20346,2416171,2.0,2,1178.0,169908.0,1779.86,1.05,6111,Ventura,CA
